## Cred_scor_url_titles_bow
## Анализ заголовков страниц в задаче кред скоринга
DMP-3643

In [1]:
##### Config
from pyspark import SparkConf, SparkContext, HiveContext
import re
import numpy as np
import pandas as pd
import datetime
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.classification import LogisticRegressionWithSGD, NaiveBayes, NaiveBayesModel
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import hashlib
from collections import Counter

hive_config_query = '''
set hive.vectorized.execution.enabled=true;
set hive.vectorized.execution.reduce.enabled = true;
set mapreduce.map.memory.mb=4096;
set mapreduce.map.child.java.opts=-Xmx4g;
set mapreduce.task.io.sort.mb=1024;
set mapreduce.reduce.child.java.opts=-Xmx4g;
set mapreduce.reduce.memory.mb=7000;
set mapreduce.reduce.shuffle.input.buffer.percent=0.5;
set mapreduce.input.fileinputformat.split.minsize=536870912;
set mapreduce.input.fileinputformat.split.maxsize=1073741824;
set hive.optimize.ppd=true;
set hive.merge.smallfiles.avgsize=536870912;
set hive.merge.mapredfiles=true;
set hive.merge.mapfiles=true;
set hive.hadoop.supports.splittable.combineinputformat=true;
set hive.exec.reducers.bytes.per.reducer=536870912;
set hive.exec.parallel=true;
set hive.exec.max.created.files=10000000;
set hive.exec.compress.output=true;
set hive.exec.dynamic.partition.mode=nonstrict;
set hive.exec.max.dynamic.partitions=1000000;
set hive.exec.max.dynamic.partitions.pernode=100000;
set io.seqfile.compression.type=BLOCK;
set mapreduce.map.failures.maxpercent=5;
'''
try:
    sc.stop()
except:
    pass
conf = (SparkConf()
        .set("spark.executor.instances", 2)
        .set("spark.driver.maxResultSize", "16g")
        .set('spark.driver.memory','16g')
        .set("spark.executor.memory", '4g')
        .set("spark.yarn.executor.memoryOverhead", 1048)
       )
sc = SparkContext(conf=conf)
hc = HiveContext(sc)

for q in hive_config_query.split(';'):
    try:
        hc.sql(q)
    except:
        pass

/opt/anaconda/lib/python2.7/site-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


### tf

In [2]:
hashingTF = HashingTF(numFeatures = 2 ** 17)
train_data = (hc.sql('select phone_num, call_ymd, approve, bow from user_kposminin.cred_scor_title_bow1 where not approve is Null limit 100')
                .filter('call_ymd < "2016-11-01"')
                .rdd
                .map(lambda row:LabeledPoint(row.approve,hashingTF.transform(row.bow)))
             )

test_data = (hc.sql('select phone_num, call_ymd, approve, bow from user_kposminin.cred_scor_title_bow1 where not approve is Null limit 100')
                .filter('call_ymd >= "2016-11-01"')
                .rdd
                .map(lambda row:LabeledPoint(row.approve,hashingTF.transform(row.bow)))
             )
train_data.cache()

PythonRDD[44] at RDD at PythonRDD.scala:48

In [22]:
#Train NaiveBayes model

modelNB = NaiveBayes.train(train_data)

def predict_proba_NB(f,model):
    import numpy as np
    '''
    Naive Bayes model prediction with probability. f is features [Sparse] vector. model is mllib.NaiveBayesModel.
    Function selects winning class with it probability.
    Output: tuple with model selected class number as first element (type int) and it probability as second (type float).
    '''
    logp = [[i,f.dot(model.theta[i]) + model.pi[i]] for i in range(len(model.theta))] # classes with log probabilities
    wi = sorted(logp, key = lambda e:  - e[1])[0][0] #winning index
    prob = 1./sum([np.exp(e[1] - logp[wi][1]) for e in logp]) #winning class probability
    return wi, prob

def predict_proba_NB_2(f, model):
    import numpy as np
    '''
    Naive Bayes model prediction with probability for 2-class classification.
    f is features [Sparse] vector. model is mllib.NaiveBayesModel.
    Output: probability of class 1 (type float).
    '''
    if len(model.theta) != 2:
        print('Model is NOT a 2-class classifier')
        return None
    logp = [f.dot(model.theta[i]) + model.pi[i] for i in range(2)]    
    return 1./(1. + np.exp(logp[0] - logp[1]))



In [42]:
#LogisticRegression model
modelLR = LogisticRegressionWithSGD.train(train_data)
modelLR.clearThreshold()

#LogisticRegression model
modelLR2 = LogisticRegressionWithSGD.train(train_data,regType = 'l1',regParam = 0.05)
modelLR2.clearThreshold()

In [43]:
print('start new mpodel')
modelLR3 = LogisticRegressionWithSGD.train(train_data,regType = 'l1',regParam = 0.01)
modelLR3.clearThreshold()

start new mpodel


In [12]:
'''
ML approach
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
hashingTF = HashingTF(inputCol="bow", outputCol="Features", numFeatures=2**17)
df_train_tf = hashingTF.transform(df_train)
'''

In [24]:
df_test = test_data.map( lambda lp: pyspark.sql.Row(
        Label = lp.label,
        NaiveBayes = float(predict_proba_NB_2(lp.features, modelNB)),
        LogisticRegression = float(modelLR.predict(lp.features))
    )).toDF().toPandas()
df_test.shape

(72719, 3)

In [44]:
df_test2 = test_data.map( lambda lp: pyspark.sql.Row(
        Label = lp.label,        
        LogisticRegression2 = float(modelLR2.predict(lp.features)),
        LogisticRegression3 = float(modelLR3.predict(lp.features))
    )).toDF().toPandas()
df_test2.shape

(72719, 3)

In [36]:
#Build AUCROC metric and print results
import sklearn
AUCROC = {}
for c in df_test.columns:
    if c!= 'Label':
        AUCROC[c] = sklearn.metrics.roc_auc_score(df_test['Label'],df_test[c])
        
print('Methods AUCROC performance on test sample ({0:.0f} samples with {1:.0f} positives):\n'.format(df_test.size,df_test['Label'].sum()) +
     '\n'.join(['{0:<30}{1:.5f}'.format(k,v) for (k,v) in AUCROC.items()]))

Methods AUCROC performance on test sample (218157 samples with 28991 positives):
LogisticRegression            0.58035
NaiveBayes                    0.61716


## TF NaiveBayes       AUC ROC             0.61716

In [48]:
#Build AUCROC metric and print results
import sklearn
AUCROC = {}
for c in df_test2.columns:
    if c!= 'Label':
        AUCROC[c] = sklearn.metrics.roc_auc_score(df_test2['Label'],df_test2[c])
        
print('Methods AUCROC performance on test sample ({0:.0f} samples with {1:.0f} positives):\n'.format(df_test2.size,df_test2['Label'].sum()) +
     '\n'.join(['{0:<30}{1:.5f}'.format(k,v) for (k,v) in AUCROC.items()]))

Methods AUCROC performance on test sample (218157 samples with 28991 positives):
LogisticRegression2           0.57517
LogisticRegression3           0.57918
